## Politicas Basadas en Gradientes

### Busqueda de Politicas (Policy Search)

- Una politica es un set de acciones que un agente debe ejecutar para maximizar los rewards a traves del tiempo.
- El algoritmo de las politicas no debe ser necesariamente deterministico.
- La idea es maximizar los rewards, sin importar el nivel de observacion del ambiente (parcialmente observable - totalmente observable)

Pensemos en una Roomba:

<img src="img/rl1.jpg" />

- la aspiradora robot se mueve solo adelante y atras
- puede rotar g grados hacia la derecha o izquiera.
- Si el algoritmo de politicas es probabilistico, entonces se puede decir que si la posibilida de moverse hacia adelante es p, entonces la probabilidad de moverse hacia atras es 1 - p. 
- si el angulo de rotacion es aleatorio, entonces se puede mover al angulo r+ o r-. 
- probablemente si ejecutamos la Roomba con esos parametros se va a mover de forma erratica, pero que importa!, lo que importa es si es efectiva la politica, osea, cuanto polvo recoge en 30 minutos.

<img src="img/rl2.png" />

Si disenamos un algoritmo para entrenar nuestra roomba, vamos a tener que avergiguar 2 paramertos: la probabilidad p de moverse hacia adelante, y el angulo de rotacion r.

Podemos usar fuerza bruta o bien la aleatoriedad para definir esos parametros y ver si se maximizan los rewards. Sin embargo esto no es solo ineficiente sino poco practico.

El concepto de **policy search** es entonces ese esfuerzo que se hace para averiguar esos parametros en un **policy space**.

Existen diversas formas de crear politicas:

- algoritmos geneticos
- algoritmos neuronales
- usar tecnicas de optimizacion: uso de gradientes. las politicas que se consiguen utilizando gradientes son llamadas **policy gradients**

### Politicas Neuronales

El problema de la politca que especificamos en el notebook anterior, es que esta no aprende, solo reacciona basado en una condicion. Podriamos reemplazar esa politica por una red neuronal perceptron, que reciba de entrada un dense de (input_shape = 4) para los parametros observados y que esta devuelva un dense (1, activacion = sigmoid) que reprenta 1 = derecha, 0 = izquierda. 

El pseudo-codigo del modelo seria algo asi:

model = sequential <br/>
&nbsp;&nbsp;&nbsp;Dense(5, input_shape = [4]),<br/>
&nbsp;&nbsp;&nbsp;Dense(1, activation = sigmoid)

Ahora bien, para entrenar esta red neuronal, podriamos que generar acciones aletorias con una probabilidad *p* de ir a la derecha y una probabilidad *1-p* de ir a la izquiera. Esto podria servir, principalmente porque permite una exploracion alatoria del ambiente. Sin embargo esta politica asi solita, no contempla el estado del paso anterior.

Debido al problema de que RL solo juzga la politica escogida por el rewards acumulados, es dificl que pasos contribuyeron de forma positiva o negativa. Este problema se llama el "Credit Assigment Problem".

Un ejemplo tipico es el del perro. Piense en un zaguate al que ud lleva todo el dia diciendole que no orine adentro de la casa. Cuando finalmente orina fuera de la casa, ud le da una galleta 10 minutos despues. Ahora sabe el perro porque lo estan premiando?

### Credit Assigment Problem

Una estrategia para lidiar con este problema consiste en evaluar la accion basado en la suma los premios que se generaron despues. Ademas de esto se aplica un factor de descuento (gamma $\gamma$) en cada paso. La suma de las acciones con descuento se llaman el "retorno" (return) de la accion. Veamos un ejemplo:

Si el agente de Pole decide ir a la derecha tres veces, este obtiene en el primer paso un premio de (+10), en el segundo paso (0) y en el tercer paso (-50), si se asume un factor de descuento $\gamma$ = 0.8, la primera accion va a tener un retorno (return) de:

- 10 + ($\gamma$ * 0) + ($\gamma^2$ * (-50)) = -22

Si $\gamma$ es cercano a cero, entonces los futuros rewards no van a contar mucho, solo los mas cercanos tendrian mas prevalencia. Normalmente se define $\gamma$ en el rango de [0.9, 0.99] pero ud es bienvenido a explorar.

El termino "Action Advantage" (AA) se refiere a que tan buena es una accion en comparacion con otras. Para estimar esto, debemos ejercutar varios episodios para estimar el AA. AL final debemos tener un buen balance de AA negativos y positivos para que el agente pueda aprender. 

Para lograr esto, vamos a desarrollar el algoritmo REINFORCE basado en politicas de gradientes. Esto lo vemos a continuacion.


### Politicas basadas en Gradientes

Algoritmo REINFORCE:

1. Nuestra politica neuronal juega el juego varias veces. En cada step, se debe calcular los gradientes que harían que la acción elegida sea aún más probable, pero no se aplican los gradientes, solo se calculan.
2. Despues de la ejecuacion de varios episodios, se calcula el "Action Advantage" de cada accion.
3. Si el AA es bueno, entonces vamos a aplicar los gradientes para que la accion sea mas probable en el futuro. Si el AA es negativo, entonces se aplican inversamente los gradientes para bajar la probabilidad de escogencia de esa accion.
4. Finalmente, se calcula la media de todos los gradientes y se aplica Gradient Descent.

El siguiente codigo, es una implementacion con Keras y Tensorflow del Algoritmo REINFORCE:

In [1]:
# conda install -c conda-forge gym
import gym
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

### Red Neuronal

- Recibe 4 inputs [car position, car velocity, pole angle, pole angular velocity]
- devuelve 1/0 para izquierda o derecha


In [3]:
# modelo que predice el proximo paso. segun lo observado.
n_inputs = 4 #

model = keras.models.Sequential([
    keras.layers.Dense(5, activation="elu", input_shape=[n_inputs]),
    keras.layers.Dense(1, activation="sigmoid")
])

### REINFORCE Utility Functions

**tf.GradientTape()**: TensorFlow proporciona la API tf.GradientTape para la diferenciación automática; es decir, calcular el gradiente de un cálculo con respecto a algunas entradas, normalmente tf.Variables. TensorFlow "registra" las operaciones relevantes ejecutadas dentro del contexto de un tf.GradientTape en una "cinta". Luego, TensorFlow usa esa cinta para calcular los gradientes de un cálculo "grabado" mediante la diferenciación en modo inverso.

In [4]:
# se calcula la probabilidad de ir a la izquierda.
# se calcula el gradiente
# se ejecuta un paso de env.step(...)
# return: se devuelve lo que genero el paso (observaciones) y el gradiente actual.
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        left_proba = model(obs[np.newaxis]) # dada una muestra, la probabilidad de ir a la izquierda
        action = (tf.random.uniform([1,1]) > left_proba) # probabilidad aleatoria contra left_proba
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32) # prob left = (1-action)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba)) # calcular la pérdida del paso actual.
    grads = tape.gradient(loss, model.trainable_variables) # almacenar gradiente en grads. (usar más tarde)
    obs, reward, done, info = env.step(int(action[0,0].numpy())) # juega la acción y obtén una nueva observación.
    return obs, reward, done, grads

In [5]:
# un episodio es un epoch
# un episodio se compone de varios steps.
# vamos a calcular los rewards del step actual.
# vamos a acumular los rewards
# vamos a acumular los gradientes.
# return: se devuelve todos los rewards acumuladors y todos los gradientes
def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        obs = env.reset()
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn) # se ejecuta un step.
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break;
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads

# esto devuelve una lista de recompensas por episodio y una lista de gradientes por episodio

In [6]:
# Recordemos que debemos descontar gamma (discount_factor) a los rewards
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    for step in range(len(rewards) -2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted

# se aplican los descuentos a los rewards y ademas se normalizan los datos.
def discount_and_normalize(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(rewards, discount_factor) for rewards in all_rewards] # aplica descuento
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std for discounted_rewards in all_discounted_rewards] # normalizacion

# prueba rapida
print(discount_rewards([10, 0, -50], discount_factor = 0.8))
print(discount_and_normalize([[10, 0, -50],[10,20]], discount_factor = 0.8))

[-22 -40 -50]
[array([-0.28435071, -0.86597718, -1.18910299]), array([1.26665318, 1.0727777 ])]


### Ajuste de hiper-parametros

In [7]:
n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_rate = 0.95
optimizer = keras.optimizers.Adam(lr=0.01)
loss_fn = keras.losses.binary_crossentropy

### Main Loop de Entrenamiento

In [14]:
# creamos el ambiente en Open AI-Gym
env = gym.make("MountainCarContinuous-v0")
env.seed(42);

# el main loop de entrenamiento
for iteration in range(n_iterations):
    
    # Ejecutamos multiples episodios los cuales tienen varios steps
    all_rewards, all_grads = play_multiple_episodes(
        env, n_episodes_per_update, n_max_steps, model, loss_fn)
    
    # acumula todos los rewards
    total_rewards = sum(map(sum, all_rewards))                    
    
    print("\rIteration: {}, mean rewards: {:.1f}".format(          
        iteration, total_rewards / n_episodes_per_update), end="")
    
    # los rewards se les aplica el descuento y se normalizan
    all_final_rewards = discount_and_normalize(all_rewards,
                                                       discount_rate)
    
    # se calcula la media ponderada de los gradientes para cada variable
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index]
             for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis=0)
        all_mean_grads.append(mean_grads)
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))

env.close()

InvalidArgumentError: Matrix size-incompatible: In[0]: [1,2], In[1]: [4,5] [Op:MatMul]

### Play trained model

In [11]:
from matplotlib.animation import FuncAnimation

def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

def render_policy_net(model, n_max_steps=500, seed=42):
    frames = []
    env = gym.make("MountainCarContinuous-v0")
    env.seed(seed)
    np.random.seed(seed)
    obs = env.reset()
    for step in range(n_max_steps):
        frames.append(env.render(mode="rgb_array"))
        left_proba = model.predict(obs.reshape(1, -1))
        action = (tf.random.uniform([1,1]) > left_proba)
        obs, reward, done, info = env.step(int(action[0,0].numpy()))
        #print(obs)
        if done:
            break
    env.close()
    return frames

In [12]:
frames = render_policy_net(model)
plot_animation(frames)

ValueError: in user code:

    /Users/cdiaz19/ml-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /Users/cdiaz19/ml-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/cdiaz19/ml-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/cdiaz19/ml-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/cdiaz19/ml-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/cdiaz19/ml-env/lib/python3.8/site-packages/tensorflow/pyth